In [1]:
# #!pip install chromadb
# %pip install -U langchain langchainhub openai --quiet

#  Initialise GenAI call

In [2]:
# GenAI Variables and models

import os

OPENAI_API_KEY = 'sk-xxx'
VERTEX_API_KEY = "xxx"

OPENAI_MODEL_VISION_PREVIEW = 'gpt-4-1106-vision-preview'
OPENAI_MODEL_GPT3 = 'gpt-3.5-turbo-0125'

VERTEX_MODEL_GEMINI = 'gemini-1.0-pro'
VERTEX_MODEL_GEMINI_VISION = 'gemini-1.0-pro-vision'

VERTEX_MODEL_ANTROPIC_HAIKU = 'claude-3-haiku@20240307'
VERTEX_MODEL_ANTROPIC_SONNET = 'claude-3-sonnet@20240229'
VERTEX_MODEL_MISTRAL = 'mistralai/Mixtral-8x7B-v0.1'
OPENAI_EMBEDDING_3_LARGE = 'text-embedding-3-large'
OPENAI_EMBEDDING_3_SMALL = 'text-embedding-3-small'
OPENAI_EMBEDDING_ADA_002 = 'text-embedding-ada-002'


os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


In [3]:
def list_file (dir:str, filter):
    return [os.path.join(root, file)  for root, dirs, files in os.walk(dir) for file in files if filter in file]

# Initilise ChromaDB

In [4]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_fn = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name=OPENAI_EMBEDDING_ADA_002)
# embedding_fn = embedding_functions.DefaultEmbeddingFunction()

class Collections:
    Name: str
    Documents: list
    Metadatas: list
    Ids: list
    count_items: int

    def __init__ (self, Name, Documents, Metadatas, Ids):
        if ( 
            len(Documents) != len (Ids) &
            len(Metadatas) != len(Documents)
        ):
            raise Exception("Error: Length of parameters do not match")
        
        self.Name = Name
        self.Documents = Documents
        self.Metadatas = Metadatas
        self.Ids = Ids
        self.count_items = len(Ids)

    def get_count(self):return self.count_items
    def get_Documents(self): return self.Documents
    def get_Metadatas(self): return self.Metadatas
    def get_Ids(self): return self.Ids
    def get_Name(self): return self.Name


class Document_vdb:
    vdb: chromadb
    collections: Collections

    def __init__ (
            self, 
            vdb = None, 
            collections = None, 
            persistent_dir = None
        ):
        if vdb == None:
            if persistent_dir == None:
                self.vdb = chromadb.Client()
            else:
                self.vdb = chromadb.PersistentClient(path = persistent_dir)
        else:
            self.vdb = vdb
        
        if collections != None:
            self.set_collections(collections)
        else: self.collections = None

    def get_vdb (self):
        return self.vdb
    
    def get_collection(
            self, 
            collection_name:str
        ):
        return self.vdb.get_collection(name = collection_name )

    def set_collections(
            self, 
            collections:Collections, 
            emb_fn = None
        ):

        if emb_fn == None: emb_fn = embedding_fn
        count_items = collections.get_count()

        collection = self.vdb.get_or_create_collection(
            name=collections.get_Name(), 
            embedding_function=emb_fn 
        )
        
        Documents = collections.get_Documents()
        Metadatas = collections.get_Metadatas()
        Ids = collections.get_Ids()

        for i in range(0, count_items):
            collection.add( 
                documents = Documents[i],
                metadatas = Metadatas[i],
                ids = Ids[i]
            )


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter
import hashlib
import uuid 
import pypdf


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

text_splitter_token = TokenTextSplitter(chunk_size=500, chunk_overlap=0)

def create_collections_dir (dir, collection_name) -> Collections:
    files = list_file(dir, ".pdf")
    Metadatas = []
    Ids = []
    Documents = []

    for file in files:
        with open(file, 'rb') as f:
            hash = str(hashlib.md5(f.read()).hexdigest())

        pdfFileObject = open(file, 'rb')
        pdfReader = pypdf.PdfReader(pdfFileObject)
        count = len(pdfReader.pages)

        for i in range(count):
            metadata = {
                'source': file,
                'page': i,
                'source_hash': hash,
            }
            page_chunks = text_splitter.split_text (pdfReader.pages[i].extract_text())
            for page_chunk in page_chunks:
                Ids.append(uuid.uuid1().hex)
                Documents.append(page_chunk)
                Metadatas.append(metadata)

    return Collections(
        Name = collection_name ,
        Documents=Documents,
        Metadatas=Metadatas,
        Ids=Ids
    )


def create_collection_token (file, collection_name) -> Collections:
    Metadatas = []
    Ids = []
    Documents = []

    with open(file, 'rb') as f:
        hash = str(hashlib.md5(f.read()).hexdigest())

    pdfFileObject = open(file, 'rb')
    pdfReader = pypdf.PdfReader(pdfFileObject)
    count = len(pdfReader.pages)

    for i in range(count):
        metadata = {
            'source': file,
            'page': i,
            'source_hash': hash,
        }
        page_chunks = text_splitter_token.split_text (pdfReader.pages[i].extract_text())
        for page_chunk in page_chunks:
            Ids.append(uuid.uuid1().hex)
            Documents.append(page_chunk)
            Metadatas.append(metadata)

    return Collections(
        Name = collection_name ,
        Documents=Documents,
        Metadatas=Metadatas,
        Ids=Ids
    )



def create_collection_char (file, collection_name) -> Collections:
    Metadatas = []
    Ids = []
    Documents = []

    with open(file, 'rb') as f:
        hash = str(hashlib.md5(f.read()).hexdigest())

    pdfFileObject = open(file, 'rb')
    pdfReader = pypdf.PdfReader(pdfFileObject)
    count = len(pdfReader.pages)

    for i in range(count):
        metadata = {
            'source': file,
            'page': i,
            'source_hash': hash,
        }
        page_chunks = text_splitter.split_text (pdfReader.pages[i].extract_text())
        for page_chunk in page_chunks:
            Ids.append(uuid.uuid1().hex)
            Documents.append(page_chunk)
            Metadatas.append(metadata)

    return Collections(
        Name = collection_name ,
        Documents=Documents,
        Metadatas=Metadatas,
        Ids=Ids
    )


In [6]:
# from langchain.embeddings import OpenAIEmbeddings


# dir = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling'
# persist_dir = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling/chromadb_THL_pages_token'
# collection_name = "Tan_Hooi_Ling"
# files = list_file( dir, ".pdf")
# no_files = len(files)
# i = 1

# for file in files:
#     print (f"===== Processing file {i} / {no_files} -- {file}")
#     FS_collection  = create_collection_token(file, collection_name=f"{collection_name}_{i}")
#     test = Document_vdb(collections= FS_collection, persistent_dir= persist_dir)
#     i = i + 1

In [7]:
# from langchain.embeddings import OpenAIEmbeddings


# dir = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling'
# persist_dir_char = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling/chromadb_THL_pages_char'
# collection_name = "Tan_Hooi_Ling"
# files = list_file( dir, ".pdf")
# no_files = len(files)
# i = 1

# for file in files:
#     print (f"===== Processing file {i} / {no_files} -- {file}")
#     FS_collection  = create_collection_char(file, collection_name=f"{collection_name}_{i}")
#     test = Document_vdb(collections= FS_collection, persistent_dir= persist_dir_char)
#     i = i + 1

# Summarising Tan Hooi Ling

In [8]:
persist_dir_char = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling/chromadb_THL_pages_char'
persist_dir_token = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling/chromadb_THL_pages_token'

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever


# OpenAIEmbeddings = OpenAIEmbeddings(api_key=os.environ.get('OPENAI_API_KEY'), model_name=OPENAI_EMBEDDING_ADA_002)
persist_dir = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling/chromadb_THL_pages_token'


llm = ChatOpenAI(model_name=OPENAI_MODEL_GPT3 , temperature=0)

client = chromadb.PersistentClient(path=persist_dir_char)  # or HttpClient()
collections = client.list_collections()
llm_results = []
customer_name = "Tan Hooi Ling"

for collection in collections:
    print (collection.name)


    vectordb = Chroma(persist_directory=persist_dir, collection_name = collection.name, embedding_function=OpenAIEmbeddings())

    retriever = vectordb.as_retriever( search_type="mmr" , search_kwargs={"k": 5, "include_metadata": True})




    retriever_from_llm = MultiQueryRetriever.from_llm( retriever=retriever, llm=llm)


    question = """

        Assume the role of a research assistant researching the biography of a person. Your current target is {customer_name}.
        You are to review the content in the text below. 

        Task: Review a document about {customer_name} and identify key biographical facts:

        1) Early Life:
            1a) Birthplace & Date
            1b) Hometown
            1c) Siblings (number)
        2) Education:
            2a) Schools attended (names & locations)
            2b) Degrees/Courses studied
            2c) Dates of attendance
        3) Career:
            3a) Employers & Positions
            3b) Employment Dates (start date and end date)


        Output:
         - Output will be markdown format.
         - Each of the 3 section will be a # Heading level 1
         - Refer to the output template in markdown format below

        <Output Template> 
        # Summary of {customer_name}
        <Place the summary of {Customer_Name} here>

        # Early Life       
        <place the {Customer_Name} early Life here in text>

        # Education 
        Output the Education background result in the following markdown format
        | School Name | Location | Course name | Degree type | Start Year | Graduation Year |
        |---|---|---|---|---|---|
        
        # Career 
        Output the career result in the following markdown format
        | Employer Name | Location | Job title | Start Year | End Year |
        |---|---|---|---|---|

        End year = refers to when the person is no longer part of the company


        contrainst:    
            Where you are unable to find the information, state them as "Not Available"
        """
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    # unique_docs = retriever_from_llm.from_llm(query=question)
    result = qa_chain({"query": question})

    llm_results.append (result['result'])

    print ("================ END OF COLLECTION 1 =================")



Tan_Hooi_Ling_4


/Users/peterwirija/Documents/GenAI/.venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of requested results 20 is greater than number of elements in index 8, updating n_results = 8


================ END OF COLLECTION 1 =================
Tan_Hooi_Ling_5


Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


================ END OF COLLECTION 1 =================
Tan_Hooi_Ling_2
================ END OF COLLECTION 1 =================
Tan_Hooi_Ling_1


Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


================ END OF COLLECTION 1 =================
Tan_Hooi_Ling_3


Number of requested results 20 is greater than number of elements in index 4, updating n_results = 4


================ END OF COLLECTION 1 =================
Tan_Hooi_Ling_6
================ END OF COLLECTION 1 =================


In [22]:

combined_results = "\n\n\n".join ([text for text in llm_results])


prompt_combine_result= rf"""
    You are summarising a set of result from LLM retrieval. The topic is the biography you like to write include {customer_name} early Life, Education, and Career
    Each result of the LLM is separated by three blank lines (\n\n\n).
    The documents were originally formatted in Markdown.and structured in the following manner 
    
    <RESULT STRUCTURE> 
    # Summary of document
    Place the summary of the document here
    
    # Summary of {customer_name}
    <Place a comprehensive summary of {customer_name} here>

    # Early Life       
    <place the {customer_name} early Life here in text>

    # Education 
    Output the Education background result in the following markdown format
    | School Name | Location | Course name | Degree type | Start Year | Graduation Year |
    |---|---|---|---|---|---|
    
    # Career 
    Output the career result in the following markdown format
    | Employer Name | Location | Job title | Start Year | End Year |
    |---|---|---|---|---|
    
    Sort the table by Employer Name

    The following contraints are only for Start Year and End Year Columns:
    - Start Year = Year of start of employment. Only output the year without month or day. Where Start Year is "Not Available" replace it with a 0. Where Start Year is "present" or "current", replace it with the current year
    - End Year = Year of end of employment. Only output the year without month or day. Where End Year is "Not Available" replace it with 0. Where End Year is "present" or "current", Replace it with the current year


    Task: 
    review the text between <RESULTS> and <\RESULTS>. 
    Combine the information for each section 
        - # Summary of {customer_name}
        - # Early Life  
        - # Education 
        - # Career 

    <RESULTS>
     {combined_results}
    <\RESULTS>

    contrainst:
        Ensure that # Summary of {customer_name} is comprehensive
        The output should retain the same structure in each section
        Do not review results that are "No information found!!"
        Do not include <RESULTS> or <\RESULTS> in the output
        Replace all "Not Available" with a 0
"""

result_summary = llm.invoke(prompt_combine_result)

In [23]:
print (result_summary.content)

# REPLACING NOT AVAIALBLE WITH 0 - RESULT SHOWS THAT THIS IS CASE SENSTIVE IN GPT3.5, but GPT4 is more likely to understand the meaning 

# Summary of Tan Hooi Ling
Tan Hooi Ling is a Malaysian businesswoman based in Singapore, best known as the co-founder and Chief Operating Officer (COO) of Grab Holdings Inc. She has been a key figure in leading the company's technology and corporate strategy teams.

# Early Life
- **Birthplace & Date:** Not Available
- **Hometown:** Kuala Lumpur, Malaysia
- **Siblings (number):** One older brother

# Education
| School Name | Location | Course name | Degree type | Start Year | Graduation Year |
|---|---|---|---|---|---|
| University of Bath | United Kingdom | Mechanical Engineering | Bachelor's Degree | 0 | 2006 |
| Harvard Business School | United States | Business Administration | Master of Business Administration | 0 | 2011 |

# Career
| Employer Name | Location | Job title | Start Year | End Year |
|---|---|---|---|---|
| McKinsey & Company | Malaysia | Business Analyst | 0 | 0 |
| Salesforce | San Francisco | Not Available | 0 | 0 |
| Grab Holdings Inc. | Singapore | Chief Operat

# Summary of Tan Hooi Ling
Tan Hooi Ling is a Malaysian businesswoman based in Singapore, best known as the co-founder and Chief Operating Officer (COO) of Grab Holdings Inc. She is stepping down from her operational roles in 2023 after leading the company's technology and corporate strategy teams. Tan Hooi Ling is known for her humble and hardworking nature.

# Early Life
- Birthplace & Date: Kuala Lumpur, Malaysia
- Hometown: Petaling Jaya, Malaysia
- Siblings: One older brother

# Education
| School Name | Location | Course name | Degree type | Start Year | End Year |
|---|---|---|---|---|---|
| University of Bath | United Kingdom | Mechanical Engineering | Bachelor's degree | 2006 | Not Available |
| Harvard Business School | United States | Business Administration | Master of Business Administration | 2011 | Not Available |

# Career
| Employer Name | Location | Job title | Start Year | End Year |
|---|---|---|---|---|
| McKinsey & Company | Malaysia | Business Analyst | Not Available | Not Available |
| Salesforce | San Francisco | Not Available | Not Available | Not Available |
| Grab Holdings Inc. | Singapore | Chief Operating Officer | 2015 | 2023 |

In [24]:
import re
import pandas as pd
def MD_to_df (text):
    lines = text.split("\n")
    header = lines[1].strip("|").split("|")

    data = [] 

    # Loop through lines starting from 2
    for line in lines[3:]:
        
        # Break once we hit an empty line
        if not line.strip():
            break
            
        cols = line.strip("|").split("|")
        row = dict(zip(header, cols))
        data.append(row)
        
    return pd.DataFrame(data)

In [26]:
import pandas as pd


result_string = str(result_summary.content)
results_content = result_string[result_string.find("# Career") + 8:]

print (results_content)

employment_history =  MD_to_df (results_content)


employment_history[' Start Year '] = employment_history[' Start Year '].astype(int)
employment_history[' End Year '] = employment_history[' End Year '].astype(int) 

employment_history = employment_history.astype({' Start Year ':'int64', ' End Year ':'int64'})

for i, years in employment_history.apply(
    lambda x: range(
        min(x[' Start Year '], x[' End Year ']),
        max(x[' Start Year '], x[' End Year ']) + 1
    ),
    axis='columns'
).items():
    for year in years:
        new_index = len(employment_history.index)
        employment_history.loc[new_index] = employment_history.loc[i].values
        employment_history.loc[new_index, 'year'] = int(year)

employment_history = employment_history.dropna().reset_index()


# employment_history.loc[(employment_history[' Start Year '] == 0) & (employment_history[' End Year '] != 0) & (employment_history['year'] != 0) ]
employment_history.drop(employment_history.loc[(employment_history[' Start Year '] == 0) & (employment_history[' End Year '] != 0) & (employment_history['year'] != 0) ].index, inplace=True)
employment_history.drop(employment_history.loc[(employment_history[' Start Year '] != 0) & (employment_history[' End Year '] == 0) & (employment_history['year'] != 0) ].index, inplace=True)
employment_history.drop('index', axis=1)


| Employer Name | Location | Job title | Start Year | End Year |
|---|---|---|---|---|
| McKinsey & Company | Malaysia | Business Analyst | 0 | 0 |
| Salesforce | San Francisco | Not Available | 0 | 0 |
| Grab Holdings Inc. | Singapore | Chief Operating Officer | 2015 | 2023 |


,Employer Name,Location,Job title,Start Year,End Year,year
0,McKinsey & Company,Malaysia,Business Analyst,0,0,0.0
1,Salesforce,San Francisco,Not Available,0,0,0.0
2,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2015.0
3,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2016.0
4,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2017.0
5,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2018.0
6,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2019.0
7,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2020.0
8,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2021.0
9,Grab Holdings Inc.,Singapore,Chief Operating Officer,2015,2023,2022.0


In [31]:
persist_dir = r'/Users/peterwirija/Documents/GenAI/Data/Tan_Hooi_Ling/chromadb_THL_pages_token'

llm = ChatOpenAI(model_name=OPENAI_MODEL_GPT3 , temperature=0)

client = chromadb.PersistentClient(path=persist_dir_char)  # or HttpClient()
collections = client.list_collections()
llm_results = []
combined_result = []

for i in range(0, len(employment_history)):
    for collection in collections:
        # text = "\n\n\n".join ([item for item in collection.get()['documents'] if "hooi ling".lower() in item.lower()])


        for text in collection.get()['documents']:
            if not("hooi ling".lower() in text.lower()): continue
            prompt_employment_SOW= rf"""
                You are an analyst. Your job is to review the text between <START OF TEXT> and <END OF TEXT> below to identify  {customer_name}'s income.
                Income is defined as any salary, shares, bonuses, and benefits. 

                {customer_name} has multiple employment. You are to gather information about a specific employment
                    Employer: {employment_history.iloc[i,0]}
                    Location: {employment_history.iloc[i,1]}
                    Job Title: {employment_history.iloc[i,2]}
                    Year of employment : {employment_history.iloc[i,5]}

                <START OF TEXT>
                {text}
                <END OF TEXT>

                Task: 
                    Review the between <START OF TEXT> and <END OF TEXT>
                    Extract key income information that {customer_name} received during {customer_name} time in {employment_history.iloc[i,0]} for the year {employment_history.iloc[i,5]}. information should include
                    - Salary
                    - Shares
                    - Bonuses
                    - Other benefits
                    Provide a narrative to {customer_name} employment in {employment_history.iloc[i,0]}

                Output:
                    Put the output in markdown format as per the following table:
                    | Employer Name | Location | Job title | Year | Salary from Employment | Bonus from Employment | Shares | Benefit received from Employment  | Narrative |

                    Markdown table Data dictionary:
                    Employer Name =  Employer name
                    Location =  location employer is located at
                    Job Title = Job title at place of employment 
                    Year = Current Year under review
                    Salary from employment  = Salary {customer_name} received for the year that {customer_name} was employed by employer
                    Bonus from Employment = Bonus  {customer_name} received for the year that {customer_name} was employed by employer
                    Shares = Shares  {customer_name} held in the company for the year
                    Benefit from Employment = Benefits, other than Salary, Bonus and shares, receiverd by {customer_name} for the year that {customer_name} was employed by employer

                contrainst:
                    Ensure that Narrative column is comprehensive
                    Do not make up information.
                    Where information is not available state "Not Available"
            """

            result = llm.invoke(prompt_employment_SOW)
            # qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

            # # unique_docs = retriever_from_llm.from_llm(query=question)
            # result = qa_chain({"query": prompt_employment_SOW})
            combined_result.append (result.content)

            print (rf"============== {i} ===============")
            print (result.content)

combined_result

============== 0 ===============
| Employer Name | Location | Job title | Year | Salary from Employment | Bonus from Employment | Shares | Benefit received from Employment  | Narrative |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| McKinsey & Company | Malaysia | Not Available | 0 | Not Available | Not Available | Not Available | Not Available | Tan Hooi Ling's employment at McKinsey & Company in Malaysia for the year 0 does not have any specific income information available in the provided text. It is unclear if Tan Hooi Ling received any salary, bonuses, shares, or benefits during this time. The text primarily focuses on her role as the co-founder and chief operating officer of Grab, a start-up she founded in 2012. Therefore, it is not possible to provide detailed income information for her employment at McKinsey & Company in Malaysia for the year 0 based on the text provided. |
============== 0 ===============
| Employer Name | Location | Job title | Year | Salary from 

In [30]:

combined_text = "\n\n\n".join ([text for text in combined_result])

OPENAI_MODEL_GPT4_0125 = 'gpt-4-0125-preview'

llm = ChatOpenAI(model_name=OPENAI_MODEL_GPT4_0125 , temperature=0)

prompt_combine_result= rf"""
    You are summarising a set of result from LLM retrieval. The topic is the biography you like to write include {customer_name} early Employment 
    Each result of the LLM is separated by three blank lines (\n\n\n).
    The documents were originally formatted in Markdown.and structured in the following manner 
    
    <RESULT STRUCTURE> 
    # Summary of document
    Place the summary of the document here
    
    
    # Career 
    Output the career result in the following markdown format
    Put the output in markdown format as per the following table:
    | Employer Name | Location | Job title | Year | Salary from Employment | Bonus from Employment | Shares | Benefit received from Employment  | Narrative |

    Markdown table Data dictionary:
    Employer Name =  Employer name
    Location =  location employer is located at
    Job Title = Job title at place of employment 
    Year = Current Year under review
    Salary from employment  = Salary {customer_name} received for the year that {customer_name} was employed by employer
    Bonus from Employment = Bonus  {customer_name} received for the year that {customer_name} was employed by employer
    Shares = Shares  {customer_name} held in the company for the year
    Benefit from Employment = Benefits, other than Salary, Bonus and shares, receiverd by {customer_name} for the year that {customer_name} was employed by employer
    
    Sort the table by Employer Name

    The following contraints are only for Start Year and End Year Columns:
    - Start Year = Year of start of employment. Only output the year without month or day. Where Start Year is "Not Available" replace it with a 0. Where Start Year is "present" or "current", replace it with the current year
    - End Year = Year of end of employment. Only output the year without month or day. Where End Year is "Not Available" replace it with 0. Where End Year is "present" or "current", Replace it with the current year


    Task: 
    review the text between <RESULTS> and <\RESULTS>. 
    Combine the information into a single row where "Employer Name", "Location", "Job Title", and "Year" are the same 

    <RESULTS>
     {combined_text}
    <\RESULTS>

    contrainst:
        Ensure that # Summary of {customer_name} is comprehensive
        The output should retain the same structure in each section
        Do not review results that are "No information found!!"
        Do not include <RESULTS> or <\RESULTS> in the output
        Replace all "Not Available" with a 0
"""

result_employment_SOW = llm.invoke(prompt_combine_result)

print(result_employment_SOW.content)

KeyboardInterrupt: 